# Documentation général : Génération d’embeddings Sentinel-2 avec CROMA-Large (TorchGeo)

Ce document décrit de manière détaillée, claire et structurée le pipeline permettant de générer des embeddings multispectraux à partir d’images Sentinel-2 en utilisant le modèle CROMA-Large de TorchGeo.  
Toutes les étapes sont expliquées, avec justification des choix techniques et références vers les sources officielles.

---

## 1. Présentation du modèle CROMA

CROMA (Cross-modal Remote Sensing Autoencoder) est un modèle de vision automatique spécialement conçu pour l’observation de la Terre.  
Il a été introduit dans l’article suivant :

- CROMA — NeurIPS 2024  
  https://arxiv.org/abs/2311.00566  
- Code officiel  
  https://github.com/antofuller/CROMA  
- Implémentation TorchGeo  
  https://torchgeo.readthedocs.io/en/stable/api/models.html#torchgeo.models.croma_large

### Caractéristiques principales

* Pré-entraînement auto-supervisé sur **1 million** d’échantillons Sentinel-1 + Sentinel-2 (dataset SSL4EO-S12).
* L’encodeur optique utilise **12 bandes Sentinel-2** (la bande cirrus B10 est exclue dans le papier comme dans TorchGeo).
* Le modèle utilise une architecture **Vision Transformer (ViT)** adaptée aux données multispectrales.
* Taille d’entrée : **120 × 120 pixels** (dimension imposée par l’architecture patch-based).
* L’encodeur produit :
  - un ensemble d'encodings par patch,
  - un embedding global pré-agrégé : **optical_GAP**, utilisé comme représentation finale.

---

## 2. Objectif du script

Le script suivant :

1. charge un ensemble d’images Sentinel-2 en GeoTIFF,
2. extrait et sélectionne les bandes attendues par CROMA (12 bandes),
3. normalise chaque bande individuellement,
4. redimensionne les images pour correspondre à l’entrée du modèle (120×120),
5. applique l’encodeur optique de CROMA-Large (pré-entraîné),
6. récupère l’embedding global **optical_GAP**,
7. applique une couche de projection optionnelle vers une dimension plus compacte (512),
8. sauvegarde tous les embeddings dans un fichier `.npy`.

---


# Documentation technique : Génération d’embeddings Sentinel-2 avec CROMA-Large (TorchGeo)

## Introduction

Ce document détaille le fonctionnement du pipeline permettant de générer des embeddings multispectraux à partir d’images Sentinel-2 en utilisant le modèle CROMA-Large fourni par la bibliothèque TorchGeo.  
L’objectif est de produire une représentation vectorielle compacte et informative d’images multispectrales pour des applications en télédétection, classification, retrieval ou intégration dans un futur modèle de langage multimodal.

Le modèle CROMA-Large utilisé ici est l’implémentation officielle proposée dans TorchGeo, s’appuyant sur les poids du dépôt original CROMA. Il s’agit d’un encodeur optique pré-entraîné sur de larges volumes de données Sentinel-2.

Sources officielles :  
- CROMA (NeurIPS 2024) : https://arxiv.org/abs/2311.00566  
- Code officiel : https://github.com/antofuller/CROMA  
- TorchGeo CROMA-Large : https://torchgeo.readthedocs.io/en/stable/api/models.html  
- Dataset SSL4EO-S12 

---

## 1. Données en entrée (Sentinel-2, 13 bandes)

Les images Sentinel-2 L2A sont fournies en 13 bandes :  
B1 à B12 + la bande cirrus B10.  
Elles sont lues depuis des fichiers GeoTIFF.  
Les valeurs sont divisées par 10000 afin de ramener la réflectance sur l’intervalle [0, 1], conformément aux recommandations ESA pour les produits Sentinel-2 L2A.

---

## 2. Sélection des 12 bandes utilisées par CROMA

Comme précisé dans l’article original, CROMA ne conserve pas la bande B10 (cirrus).  
Cette bande est très bruitée et n’est pas utilisée dans SSL4EO-S12.  
TorchGeo suit strictement cette configuration.

Les 12 bandes conservées sont donc :  
B1 à B9 (sauf B10) puis B11 et B12.

Cela garantit la compatibilité parfaite avec le modèle pré-entraîné.

---

## 3. Normalisation par bande

Chaque bande est normalisée de manière indépendante selon :

- moyenne sur l’ensemble du dataset,
- écart-type sur l’ensemble du dataset.

La normalisation per-band est indispensable pour stabiliser les représentations multispectrales, réduire les différences dynamiques entre bandes et permettre une convergence cohérente des blocs Transformer.  
Elle suit les pratiques classiques en traitement Sentinel-2.

---

## 4. Redimensionnement à 120 × 120

CROMA a été pré-entraîné sur des patches Sentinel-2 recadrés spécifiquement en **120 × 120 pixels**, ce qui correspond à sa géométrie interne (patch size = 8 → 15 × 15 patches).  
TorchGeo impose la même contrainte.

L’interpolation bilinéaire est utilisée avec :

align_corners = False

Ce paramètre garantit que les distances entre pixels sont conservées lors du redimensionnement, évitant les distorsions spatiales en bordure. C’est le paramètre recommandé par PyTorch pour les architectures modernes (ViT, MAE, CROMA).

---

## 5. Chargement du modèle CROMA-Large (optical-only)

Le modèle est initialisé par :

croma_large(modalities=[“optical”])

Cela désactive l’encodeur radar et ne garde que l’encodeur optique Sentinel-2.

CROMA-Large utilise un Vision Transformer de dimension d’encodage :

- encoder_dim = 1024

Le modèle renvoie un dictionnaire contenant plusieurs sorties, dont :

- optical_encodings : encodings par patch
- optical_GAP : embedding global (Global Average Pooling)

Le vecteur optical_GAP est la représentation la plus pertinente pour un pipeline d’embeddings.

---

## 6. Utilisation de optical_GAP comme embedding multispectral global

La sortie optical_GAP est un vecteur de dimension 1024.  
Il s’agit du pooling global des représentations par patch.  
Cette représentation a été explicitement conçue dans le papier CROMA pour être :

- robuste à la variation géographique,  
- robuste à la saisonnalité,  
- représentative du contenu spectral et spatial,  
- stable pour des tâches downstream (classification, segmentation, retrieval).

Dans le papier CROMA, ce vecteur est évalué sur divers benchmarks tels que :  
- BigEarthNet-S2,  
- BigEarthNet-MM,  
- DynamicWorld,  
- DFC2020,  
montrant qu’il surpasse les méthodes multispectrales précédentes.

---

## 7. Couche de projection (1024 → 512)

Une couche linéaire est appliquée afin de réduire la dimension de 1024 à 512.  
Cette réduction présente plusieurs avantages :

- réduction mémoire lors du stockage des embeddings,  
- amélioration de la vitesse dans les bases vectorielles (ex. FAISS),  
- standardisation vers une dimension courante pour des modèles multimodaux.

Cette étape n’altère pas l’information essentielle, car la rupture est linéaire et le modèle CROMA produit des embeddings redondants permettant une compression modérée.

---

## 8. Génération des embeddings

Le pipeline génère les embeddings en batchs pour optimiser l’utilisation du GPU.  
Pour chaque batch :

1. L’image est passée dans CROMA (optical-only).
2. La sortie optical_GAP est extraite.
3. L’embedding est projeté en 512 dimensions.
4. L’embedding est ajouté à une liste.

Les embeddings finaux sont concaténés et sauvegardés dans un fichier `.npy`.

Ce format est idéal pour une intégration ultérieure dans un pipeline de machine learning, un moteur de similarité d’images, ou un module de retrieval destiné à un LLM.

---

## 9. Utilisation des embeddings dans un futur LLM

Les embeddings issus de CROMA-Large sont adaptés pour :

- représenter de manière compacte et fidèle le contenu multispectral Sentinel-2,
- permettre la recherche de parcelles similaires,
- servir de features dans un modèle supervisé (classification, régression agronomique),
- être intégrés dans un pipeline RAG géospatial pour un LLM.

CROMA n’est pas un modèle vision-langage, mais il est parfaitement adapté pour fournir au LLM des informations d’état de la parcelle.  
L’intégration se fait typiquement via un espace vectoriel FAISS ou un adapter linéaire.

---

## Conclusion

Ce pipeline exploite fidèlement la version officielle du modèle CROMA-Large proposée dans TorchGeo, en respectant :

- les bandes Sentinel-2 utilisées lors du pré-entraînement,  
- la normalisation multispectrale,  
- les dimensions d’entrée requises,  
- les sorties officielles du modèle,  
- la préparation d’un embedding propre pour un pipeline IA moderne.

Il fournit ainsi une base solide pour la création d’un assistant agricole, d’un moteur de recommandation, ou d’un système de retrieval géospatial.  

In [ ]:
import glob
import rasterio
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchgeo.models import croma_large

# ============ DEVICE ============
device = (
    torch.device("cuda") if torch.cuda.is_available()
    else torch.device("cpu")
)

# ===============================
# 1) Load Sentinel-2 images
# ===============================
paths = glob.glob("AnnualCrop/*.tif")

images = []
for p in paths:
    with rasterio.open(p) as src:
        x = src.read().astype(np.float32) / 10000.0   # (13,H,W)
        images.append(x)

X = np.array(images, dtype=np.float32)   # (N,13,H,W)

# ===============================
# 2) KEEP 12 BANDS EXPECTED BY CROMA
# Remove B10 (index 9)
# ===============================
keep_idx = list(range(9)) + [10, 11, 12]
X = X[:, keep_idx, :, :]      # (N,12,H,W)

# ===============================
# 3) Normalization PER BAND
# ===============================
mean = X.mean(axis=(0, 2, 3), keepdims=True)
std  = X.std(axis=(0, 2, 3), keepdims=True)
X = (X - mean) / (std + 1e-9)

# ===============================
# 4) Resize → 120×120 expected by CROMA
# ===============================
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = F.interpolate(X_tensor, size=(120, 120), mode="bilinear", align_corners=False)

# ===============================
# 5) Load CROMA-Large pretrained (OPTICAL ONLY)
# ===============================
model = croma_large(modalities=["optical"]).to(device)
model.eval()

# ===============================
# 7) Compute embeddings
# ===============================
batch_size = 16
embeddings = []

with torch.no_grad():
    for i in range(0, len(X_tensor), batch_size):
        batch = X_tensor[i:i+batch_size].to(device)

        # ---- OUTPUT = dict with "optical_GAP" key ----
        output = model(x_optical=batch)
        feats = output["optical_GAP"]       # correct key

        embeddings.append(feats.cpu().numpy())

embeddings = np.vstack(embeddings)
np.save("sentinel_embeddings_1024.npy", embeddings)